In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils
from matplotlib import pyplot as plt
import os
import cv2
from collections import defaultdict
#from PIL import Image

np.random.seed(123) # for testing

image_folder_path = r'C:\Users\Tasli\Desktop\free-spoken-digit-dataset-master\Spectros'

training_data = []
training_specs = []
testing_specs = []

training_labels = []
testing_labels = []


file_paths = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f)) and '.png' in f]


Using TensorFlow backend.


In [2]:
for file_name in file_paths:
    spec_index = file_name.rsplit("_", 1)[1].rsplit(".", 1)[0]
    label = file_name[0]
    #spectrogram = Image.open(image_folder_path + '/' + file_name)
    #spectrogram = spectrogram.convert('RGB')
    #spectrogram = spectrogram.resize((224,224))
    spectrogram = cv2.imread(os.path.join(image_folder_path,file_name))
    spectrogram = cv2.resize(spectrogram, (224, 224)) 
    spectrogram = np.array(spectrogram)
    if int(spec_index) <= 4:
        testing_specs.append(spectrogram)
        testing_labels.append(label)
    else:
        training_data.append([spectrogram,label])

#shuffle the training data, then add to the lists
import random
random.shuffle(training_data)
for features,label in training_data:
    training_specs.append(features)
    training_labels.append(label)

training_specs = np.array(training_specs)
testing_specs = np.array(testing_specs)
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)


training_specs = training_specs.astype('float32') # convert types to float
testing_specs = testing_specs.astype('float32')
training_specs /= 255 # normalize data to a 0-1 value
testing_specs /= 255


training_labels = np_utils.to_categorical(training_labels, 10) # convert data to categories
testing_labels = np_utils.to_categorical(testing_labels, 10)


In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=training_specs.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(10))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(training_specs, training_labels, batch_size=32, epochs=4, validation_split=0.1)

Train on 1620 samples, validate on 180 samples
Epoch 1/4
1620/1620 [==============================] - 685s 423ms/sample - loss: 4.4286 - accuracy: 0.1000 - val_loss: 3.5120 - val_accuracy: 0.0889
Epoch 2/4
1620/1620 [==============================] - 743s 459ms/sample - loss: 3.5979 - accuracy: 0.1012 - val_loss: 3.5120 - val_accuracy: 0.0889
Epoch 3/4
1620/1620 [==============================] - 596s 368ms/sample - loss: 3.5979 - accuracy: 0.1012 - val_loss: 3.5120 - val_accuracy: 0.0889
Epoch 4/4
1620/1620 [==============================] - 582s 359ms/sample - loss: 3.5979 - accuracy: 0.1012 - val_loss: 3.5120 - val_accuracy: 0.0889


In [4]:
#Test the results
val_loss, val_acc = model.evaluate(testing_specs, testing_labels, verbose=0)

print(val_loss, val_acc)

3.5893116760253907 0.1
